# GATK Tutorial | Convolutional Neural Network (CNN) Filtering | March 2019


This GATK tutorial corresponds to a section of the GATK Workshop _2b. Germline Convolutional Neural Network (CNN) Filtering Tutorial_ worksheet. The goal is to become familiar with using Convolutional Neural Net to filter annotated variants. The notebook illustrates the following steps. 

- Use GATK to annotate a VCF with scores from a Convolutional Neural Network (CNN)
- Generate 1D and 2D CNN models
- Apply tranche filtering to VCF based on scores from an annotation in the INFO field  
- Calculate concordance metrics

### First, make sure the notebook is using a Python 3 kernel in the top right corner.
A kernel is a _computational engine_ that executes the code in the notebook. We use Python 3 in this notebook to execute GATK commands using _Python Magic_ (`!`). Later we will switch to another notebook to do some plotting in R.

### How to run this notebook:
- **Click to select a gray cell and then pressing SHIFT+ENTER to run the cell.**

- **Write results to `/home/jupyter-user/CNN/Output/`. To access the directory, click on the upper-left jupyter icon.**

### Enable reading Google bucket data 

In [1]:
!gsutil ls gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/
!gsutil ls gs://broad-references/hg19/v0/

gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/
gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/bams/
gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/references/
gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/
gs://broad-references/hg19/v0/1000G_omni2.5.b37.vcf.gz
gs://broad-references/hg19/v0/1000G_omni2.5.b37.vcf.gz.tbi
gs://broad-references/hg19/v0/1000G_phase1.snps.high_confidence.b37.vcf.gz
gs://broad-references/hg19/v0/1000G_phase1.snps.high_confidence.b37.vcf.gz.tbi
gs://broad-references/hg19/v0/Axiom_Exome_Plus.genotypes.all_populations.poly.vcf.gz
gs://broad-references/hg19/v0/Axiom_Exome_Plus.genotypes.all_populations.poly.vcf.gz.tbi
gs://broad-references/hg19/v0/Homo_sapiens_assembly19.cdna.all.fa
gs://broad-references/hg19/v0/Homo_sapiens_assembly19.cds.all.fa
gs://broad-references/hg19/v0/Homo_sapiens_assembly19.cloud_references.json
gs://broad-references/hg19/v0/Homo_sapiens_assembly19.contam.UD
gs://broad-ref

In [2]:
# If you do not see gs:// URLs listed above, run this cell to install Google Cloud Storage. 
# Afterwards, restart the kernel with Kernel > Restart.
#! pip install google-cloud-storage

In [5]:
# Write results to /home/jupyter-user/3-somatic-cna/sandbox/. 
# To access the directory, click on the upper-left jupyter icon.
!mkdir -p /home/jupyter-user/CNN/Output/

---
## Run the default 1D model on the VCF with CNNScoreVariants

CNNScoreVariant is a pre-trained Convolutional Neural Network tool to score variants. This tool uses machine learning to differentiate between good variants and artifacts of the sequencing process, a fairly new approach that is especially effective at correctly calling indels. 

> **VQSR and Hard-filtering only takes into account variant annotations. However, CNNScoreVariants 1D Model evaluates a) annotations AND b) reference files, +-64bases from the variant. Example: it accounts for regions in the ref file that RE difficult to sequence.**

To enable the models to accurately filter and score variants from VCF files, we trained on validated VCFs (from truth models including **SynDip, Genomes in a bottle, and Platinum Genomes**) with unvalidated VCFs aligned to different reference builds (**HG19, HG38**), sequenced on different machines, using different protocols. 

In [ ]:
!gatk CNNScoreVariants \
-V gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz \
-O /home/jupyter-user/CNN/Output/my_1d_cnn_scored.vcf \
-R gs://broad-references/hg19/v0/Homo_sapiens_assembly19.fasta

Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar CNNScoreVariants -V gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz -O /home/jupyter-user/CNN/Output/my_1d_cnn_scored.vcf -R gs://broad-references/hg19/v0/Homo_sapiens_assembly19.fasta
05:28:55.239 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:28:56.595 INFO  CNNScoreVariants - ------------------------------------------------------------
05:28:56.595 INFO  CNNScoreVariants - The Genome Analysis Toolkit (GATK) v4.1.0.0
05:28:56.595 INFO  CNNScoreVariants - For support and documentation go to https://software.broadinstitute.org/gat

The output VCF my_1d_cnn_scored.vcf will now have an INFO  field CNN_1D which corresponds to the score assigned by 1D model.


In [ ]:
!cat /home/jupyter-user/CNN/Output/my_1d_cnn_scored.vcf | grep -v '##' | head -5

## Apply filters to the VCF based on the CNN_1D score with the FilterVariantTranches tool

After scoring, you can filter your VCF by applying a sensitivity threshold with the tool FilterVariantTranches. 

In [8]:
!gatk FilterVariantTranches \
-V /home/jupyter-user/CNN/Output/my_1d_cnn_scored.vcf \
--resource gs://broad-references/hg19/v0/1000G_omni2.5.b37.vcf.gz \
--resource gs://broad-references/hg19/v0/hapmap_3.3.b37.vcf.gz \
--info-key CNN_1D \
--snp-tranche 95.9 \
--indel-tranche 95.0 \
-O /home/jupyter-user/CNN/Output/my_1d_filtered.vcf \
--invalidate-previous-filters 


Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar FilterVariantTranches -V /home/jupyter-user/CNN/Output/my_1d_cnn_scored.vcf --resource gs://broad-references/hg19/v0/1000G_omni2.5.b37.vcf.gz --resource gs://broad-references/hg19/v0/hapmap_3.3.b37.vcf.gz --info-key CNN_1D --snp-tranche 95.9 --indel-tranche 95.0 -O /home/jupyter-user/CNN/Output/my_1d_filtered.vcf --invalidate-previous-filters
05:30:53.802 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:30:54.005 INFO  FilterVariantTranches - ------------------------------------------------------------
05:30:54.005 INFO  FilterVariantTranches - The Genome Analysis Toolkit (GATK) v4.1

> Now you have neural network filtered VCF!

## Run the default 2D model on the VCF with CNNScoreVariants

The process is quite similar for the 2D model except we will also need to supply a BAM file with DNA read data to CNNScoreVariants.  We tell the tool to use the 2D read processing model with the tensor-type argument.

> **CNNScoreVariants 2D Model evaluates a) annotations, b) reference files and c) all variant information from the bam file.**

In [9]:
!gatk CNNScoreVariants \
-I gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/bams/g94982_chr20_1m_10m_bamout.bam \
-V gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_895.vcf \
-R gs://broad-references/hg19/v0/Homo_sapiens_assembly19.fasta \
-O /home/jupyter-user/CNN/Output/my_2d_cnn_scored.vcf \
--tensor-type read_tensor \
--transfer-batch-size 8 \
--inference-batch-size 8

Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar CNNScoreVariants -I gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/bams/g94982_chr20_1m_10m_bamout.bam -V gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_895.vcf -R gs://broad-references/hg19/v0/Homo_sapiens_assembly19.fasta -O /home/jupyter-user/CNN/Output/my_2d_cnn_scored.vcf --tensor-type read_tensor --transfer-batch-size 8 --inference-batch-size 8
05:31:08.815 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:31:09.048 INFO  CNNScoreVariants - ------------------------------------------------------------
05:31:09.048 INFO  CNNSc

## Now apply filters to the VCF based on the CNN_2D score with the FilterVariantTranches tool

In [10]:
!gatk FilterVariantTranches \
-V /home/jupyter-user/CNN/Output/my_2d_cnn_scored.vcf \
--resource gs://broad-references/hg19/v0/1000G_omni2.5.b37.vcf.gz \
--resource gs://broad-references/hg19/v0/hapmap_3.3.b37.vcf.gz \
--info-key CNN_2D \
--snp-tranche 95.9 \
--indel-tranche 95.0 \
-O /home/jupyter-user/CNN/Output/my_2d_filtered.vcf \
--invalidate-previous-filters

Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar FilterVariantTranches -V /home/jupyter-user/CNN/Output/my_2d_cnn_scored.vcf --resource gs://broad-references/hg19/v0/1000G_omni2.5.b37.vcf.gz --resource gs://broad-references/hg19/v0/hapmap_3.3.b37.vcf.gz --info-key CNN_2D --snp-tranche 95.9 --indel-tranche 95.0 -O /home/jupyter-user/CNN/Output/my_2d_filtered.vcf --invalidate-previous-filters
05:33:17.980 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:33:18.214 INFO  FilterVariantTranches - ------------------------------------------------------------
05:33:18.215 INFO  FilterVariantTranches - The Genome Analysis Toolkit (GATK) v4.1

## Evaluate the 2D Model
Now let’s evaluate how the filter did by running the concordance tool. 

In [11]:
!gatk Concordance \
-truth gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz \
-eval /home/jupyter-user/CNN/Output/my_2d_filtered.vcf \
-L 20:1000000-1432828 \
-S /home/jupyter-user/CNN/Output/2d_filtered_concordance.txt


Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar Concordance -truth gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz -eval /home/jupyter-user/CNN/Output/my_2d_filtered.vcf -L 20:1000000-1432828 -S /home/jupyter-user/CNN/Output/2d_filtered_concordance.txt
05:33:32.262 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:33:32.509 INFO  Concordance - ------------------------------------------------------------
05:33:32.510 INFO  Concordance - The Genome Analysis Toolkit (GATK) v4.1.0.0
05:33:32.510 INFO  Concordance - For support and documentation go to https://software.broadinstitute.org/

## Evaluate the unfiltered VCF

In [12]:
!gatk Concordance \
-truth gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz \
-eval gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_895.vcf \
-L 20:1000000-1432828 \
-S /home/jupyter-user/CNN/Output/unfiltered_2d_concordance.txt

Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar Concordance -truth gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz -eval gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_895.vcf -L 20:1000000-1432828 -S /home/jupyter-user/CNN/Output/unfiltered_2d_concordance.txt
05:33:47.361 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:33:47.580 INFO  Concordance - ------------------------------------------------------------
05:33:47.580 INFO  Concordance - The Genome Analysis Toolkit (GATK) v4.1.0.0
05:33:47.581 INFO  Concordance - For support and document

> Now look at how precision goes up (and sensitivity goes down) as we filter.

In [14]:
!cat /home/jupyter-user/CNN/Output/unfiltered_2d_concordance.txt

type	true-positive	false-positive	false-negative	sensitivity	precision
SNP	699	57	0	1.0	0.9246031746031746
INDEL	86	53	0	1.0	0.6187050359712231


In [13]:
!cat /home/jupyter-user/CNN/Output/2d_filtered_concordance.txt

type	true-positive	false-positive	false-negative	sensitivity	precision
SNP	668	32	31	0.9556509298998569	0.9542857142857143
INDEL	12	2	74	0.13953488372093023	0.8571428571428571


## Evaluate the 1D Model 

In [15]:
!gatk Concordance \
-truth gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz \
-eval /home/jupyter-user/CNN/Output/my_1d_filtered.vcf \
-L 20:1000000-9467292 \
-S /home/jupyter-user/CNN/Output/my_1d_filtered_concordance.txt


Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar Concordance -truth gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz -eval /home/jupyter-user/CNN/Output/my_1d_filtered.vcf -L 20:1000000-9467292 -S /home/jupyter-user/CNN/Output/my_1d_filtered_concordance.txt
05:34:06.061 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:34:06.289 INFO  Concordance - ------------------------------------------------------------
05:34:06.289 INFO  Concordance - The Genome Analysis Toolkit (GATK) v4.1.0.0
05:34:06.289 INFO  Concordance - For support and documentation go to https://software.broadinstitute.o

## Evaluate the unfiltered VCF

In [16]:
!gatk Concordance \
-truth gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz \
-eval gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz \
-L 20:1000000-9467292 \
-S /home/jupyter-user/CNN/Output/unfiltered_concordance.txt


Using GATK jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar Concordance -truth gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/hg001_na12878_b37_truth.vcf.gz -eval gs://gatk-tutorials/workshop_1903/2-germline/CNNScoreVariants/vcfs/g94982_b37_chr20_1m_15871.vcf.gz -L 20:1000000-9467292 -S /home/jupyter-user/CNN/Output/unfiltered_concordance.txt
05:34:22.592 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/etc/gatk-4.1.0.0/gatk-package-4.1.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
05:34:22.944 INFO  Concordance - ------------------------------------------------------------
05:34:22.945 INFO  Concordance - The Genome Analysis Toolkit (GATK) v4.1.0.0
05:34:22.945 INFO  Concordance - For support and docume

> **Now look at how precision goes up (and sensitivity goes down) as we filter.**

In [18]:
!cat /home/jupyter-user/CNN/Output/unfiltered_concordance.txt

type	true-positive	false-positive	false-negative	sensitivity	precision
SNP	10715	1218	558	0.9505011975516722	0.8979301097796027
INDEL	1617	1022	109	0.9368482039397451	0.6127320954907162


In [17]:
!cat /home/jupyter-user/CNN/Output/my_1d_filtered_concordance.txt

type	true-positive	false-positive	false-negative	sensitivity	precision
SNP	10576	1132	697	0.9381708507052249	0.9033139733515545
INDEL	1582	691	144	0.9165701042873696	0.6959964804223493


> Finally, you can train your own models with the tools CNNVariantWriteTensors and CNNVariantTrain, as long as you have validated VCFs to use as training data.